# Tutorial 02: Tool Calling & ReAct Agent

In this tutorial, you'll build a ReAct (Reasoning + Acting) agent from scratch. This pattern is the foundation of most LLM agents.

**What you'll learn:**
- How to define **tools** that your agent can use
- **Binding tools** to your LLM
- **Conditional edges** for routing based on LLM decisions
- The **ReAct loop**: Agent → Tools → Agent → ...
- Building an agent that can take real actions

By the end, you'll have an agent that can reason about problems and use tools to solve them.

## The ReAct Pattern

ReAct stands for **Re**asoning and **Act**ing. This pattern, introduced in the paper [ReAct: Synergizing Reasoning and Acting in Language Models](https://arxiv.org/abs/2210.03629), is the foundation of most LLM agents.

### Why ReAct?

Traditional LLMs can only generate text. But what if we want them to:
- Search the web for current information?
- Perform calculations?
- Access databases?
- Take real-world actions?

ReAct solves this by giving LLMs access to **tools** and letting them **decide** when to use them.

### The Loop

1. **User Query** → Agent receives a question or task
2. **Reasoning** → Agent thinks about what to do (may generate internal thoughts)
3. **Action** → Agent decides to call a tool OR respond to user
4. **Observation** → Tool returns results
5. **Repeat** → Back to reasoning with new information

This continues until the agent has enough information to answer.

### Visual Overview

The graph visualization below (generated in Step 8) shows this loop:
- **agent** node: Calls the LLM to reason and decide
- **tools** node: Executes requested tools
- Conditional edge: Routes based on whether tools were called

In [ ]:
# Setup: Verify Ollama connection
from langgraph_ollama_local import LocalAgentConfig

config = LocalAgentConfig()
print(f"Ollama: {config.ollama.base_url}")
print(f"Model: {config.ollama.model}")

## Step 1: Define Tools

Tools are functions that your agent can call. Each tool needs:
- A clear **name**
- A **docstring** describing what it does (the LLM reads this!)
- **Type hints** for arguments

Let's create some simple tools:

In [ ]:
from langchain_core.tools import tool

@tool
def add(a: float, b: float) -> float:
    """Add two numbers together.
    
    Args:
        a: First number
        b: Second number
        
    Returns:
        The sum of a and b
    """
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers together.
    
    Args:
        a: First number
        b: Second number
        
    Returns:
        The product of a and b
    """
    return a * b

@tool
def get_weather(location: str) -> str:
    """Get the current weather for a location.
    
    Args:
        location: City name or location
        
    Returns:
        Weather description for the location
    """
    # Mock implementation - in real use, call a weather API
    weather_data = {
        "san francisco": "Sunny, 68°F",
        "new york": "Cloudy, 55°F",
        "london": "Rainy, 50°F",
    }
    location_lower = location.lower()
    for city, weather in weather_data.items():
        if city in location_lower:
            return f"Weather in {location}: {weather}"
    return f"Weather data not available for {location}"

# Collect our tools
tools = [add, multiply, get_weather]

# Let's see what the tools look like
for t in tools:
    print(f"Tool: {t.name}")
    print(f"  Description: {t.description}")
    print()

## Step 2: Create LLM with Tool Binding

We need to tell the LLM about our tools. This is done with `bind_tools()`:

In [ ]:
from langchain_ollama import ChatOllama

# Create base LLM
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0,  # Deterministic for tool calling
)

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

print("LLM configured with tools!")

### Test Tool Calling

Let's see how the LLM decides to call tools:

In [ ]:
# Test: Does the LLM want to call a tool?
response = llm_with_tools.invoke("What is 25 times 4?")

print(f"Response type: {type(response).__name__}")
print(f"Content: {response.content}")
print(f"Tool calls: {response.tool_calls}")

## Step 3: Define State

Our agent state is the same as the chatbot - we track messages:

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    """State for our ReAct agent."""
    messages: Annotated[list, add_messages]

## Step 4: Define the Agent Node

The agent node calls the LLM and returns its response. The LLM might respond with text OR request a tool call:

In [ ]:
from langchain_core.messages import SystemMessage

# System prompt to guide the agent
SYSTEM_PROMPT = """You are a helpful assistant with access to tools.

When you need to perform calculations or get information, use the available tools.
Always explain your reasoning before and after using tools.

Available tools:
- add: Add two numbers
- multiply: Multiply two numbers  
- get_weather: Get weather for a location
"""

def agent_node(state: AgentState) -> dict:
    """Call the LLM to decide what to do next.
    
    The LLM will either:
    1. Return a text response (done)
    2. Request one or more tool calls (continue)
    """
    # Prepend system message
    messages = [SystemMessage(content=SYSTEM_PROMPT)] + state["messages"]
    
    # Call LLM with tools
    response = llm_with_tools.invoke(messages)
    
    return {"messages": [response]}

## Step 5: Define the Tool Node

The tool node executes tool calls requested by the LLM:

In [ ]:
import json
from langchain_core.messages import ToolMessage

# Create a lookup dictionary for tools
tools_by_name = {tool.name: tool for tool in tools}

def tool_node(state: AgentState) -> dict:
    """Execute tool calls from the last message.
    
    This node:
    1. Gets tool calls from the last AI message
    2. Executes each tool
    3. Returns ToolMessages with results
    """
    outputs = []
    
    # Get the last message (should be an AI message with tool calls)
    last_message = state["messages"][-1]
    
    # Execute each tool call
    for tool_call in last_message.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool_id = tool_call["id"]
        
        print(f"  Executing: {tool_name}({tool_args})")
        
        # Get the tool and execute it
        tool = tools_by_name[tool_name]
        result = tool.invoke(tool_args)
        
        print(f"  Result: {result}")
        
        # Create a ToolMessage with the result
        outputs.append(
            ToolMessage(
                content=json.dumps(result),
                name=tool_name,
                tool_call_id=tool_id,
            )
        )
    
    return {"messages": outputs}

## Step 6: Define Conditional Routing

The key to ReAct is deciding **when to stop**. After the agent runs:
- If it requested tool calls → route to tools
- If it responded with text → we're done

In [ ]:
from langgraph.graph import END

def should_continue(state: AgentState) -> str:
    """Decide whether to continue to tools or end.
    
    Returns:
        "tools" if agent requested tool calls
        "end" if agent is done (no tool calls)
    """
    last_message = state["messages"][-1]
    
    # Check if the LLM made tool calls
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    
    # No tool calls means we're done
    return "end"

## Step 7: Build the Graph

Now we assemble everything into a graph with conditional edges:

In [ ]:
from langgraph.graph import StateGraph, START, END

# Create the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)

# Add edges
# START -> agent: Begin with the agent
workflow.add_edge(START, "agent")

# agent -> conditional: Either go to tools or end
workflow.add_conditional_edges(
    "agent",           # From node
    should_continue,   # Condition function
    {
        "tools": "tools",  # If "tools", go to tools node
        "end": END,        # If "end", finish
    }
)

# tools -> agent: After tools, always go back to agent
workflow.add_edge("tools", "agent")

# Compile
graph = workflow.compile()

print("ReAct agent compiled!")

## Step 8: Visualize the Graph

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not render graph: {e}")
    print(graph.get_graph().draw_ascii())

## Step 9: Run the Agent!

Let's test our ReAct agent with different queries:

In [ ]:
def run_agent(query: str):
    """Run the agent and print the conversation."""
    print(f"\n{'='*60}")
    print(f"User: {query}")
    print("="*60)
    
    # Stream the response
    for event in graph.stream(
        {"messages": [("user", query)]},
        stream_mode="values"
    ):
        last_message = event["messages"][-1]
        
        # Print based on message type
        if hasattr(last_message, 'type'):
            if last_message.type == "ai":
                if last_message.tool_calls:
                    print(f"\nAgent deciding to use tools...")
                    for tc in last_message.tool_calls:
                        print(f"  → {tc['name']}({tc['args']})")
                elif last_message.content:
                    print(f"\nAgent: {last_message.content}")
            elif last_message.type == "tool":
                print(f"  Tool result: {last_message.content}")

# Test 1: Simple math
run_agent("What is 25 times 4?")

In [ ]:
# Test 2: Multi-step calculation
run_agent("What is 15 plus 27, then multiply the result by 3?")

In [ ]:
# Test 3: Weather query
run_agent("What's the weather like in San Francisco?")

In [ ]:
# Test 4: No tool needed
run_agent("What is the capital of France?")

## Complete Code

Here's the complete ReAct agent in one cell:

In [ ]:
# Complete ReAct Agent Implementation

import json
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, ToolMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph_ollama_local import LocalAgentConfig

# === Tools ===
@tool
def add(a: float, b: float) -> float:
    """Add two numbers together."""
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers together."""
    return a * b

tools = [add, multiply]
tools_by_name = {t.name: t for t in tools}

# === LLM ===
config = LocalAgentConfig()
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0,
).bind_tools(tools)

# === State ===
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

# === Nodes ===
def agent_node(state: AgentState):
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

def tool_node(state: AgentState):
    outputs = []
    for tc in state["messages"][-1].tool_calls:
        result = tools_by_name[tc["name"]].invoke(tc["args"])
        outputs.append(ToolMessage(
            content=json.dumps(result),
            name=tc["name"],
            tool_call_id=tc["id"],
        ))
    return {"messages": outputs}

# === Routing ===
def should_continue(state: AgentState):
    last = state["messages"][-1]
    if hasattr(last, "tool_calls") and last.tool_calls:
        return "tools"
    return "end"

# === Graph ===
workflow = StateGraph(AgentState)
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, {"tools": "tools", "end": END})
workflow.add_edge("tools", "agent")
graph = workflow.compile()

# === Test ===
result = graph.invoke({"messages": [("user", "What is 7 times 8?")]})
print(result["messages"][-1].content)

## Key Concepts Recap

| Concept | Description |
|---------|-------------|
| **Tool** | A function decorated with `@tool` that the LLM can call |
| **bind_tools()** | Tells the LLM what tools are available |
| **tool_calls** | List of tool invocations requested by the LLM |
| **ToolMessage** | Message containing a tool's output |
| **Conditional Edge** | Edge that routes based on a condition function |
| **ReAct Loop** | Agent → Tools → Agent cycle until done |

## Common Issues with Local LLMs

### Tool Calling Not Working?

Not all local models support tool calling. Check the [Ollama Tools Models](https://ollama.com/search?c=tools) page for the official list.

**Models that work well:**
- `llama3.1:8b` or larger - Best overall for function calling
- `llama3.2:3b` - Good for resource-constrained environments
- `mistral:7b` - Efficient and reliable
- `qwen3` - Featured in official Ollama docs
- `granite4` - Tool-optimized by IBM

### Inconsistent Results?

- Set `temperature=0` for deterministic tool calling
- Use explicit, detailed tool descriptions
- Add examples to your system prompt

## What's Next?

In [Tutorial 03: Memory & Persistence](03_memory_persistence.ipynb), you'll learn:
- How to persist conversations across sessions
- Using checkpointers (MemorySaver, SqliteSaver)
- Thread IDs for multi-user support
- Inspecting conversation history